# AI画像生成入門 - Google Colab版

このノートブックでは、最新のAI画像生成技術を体験します。

## 学習内容
1. テキストから画像を生成（Text-to-Image）
2. 画像の編集・加工（Image-to-Image）
3. プロンプトエンジニアリングの基礎
4. 生成画像の保存と活用

## 必要な環境
- Google Colabの無料版でも動作（GPU推奨）
- 実行時間: 約30分

In [ ]:
# 必要なライブラリのインストール
!pip install diffusers transformers accelerate torch pillow numpy
!pip install gradio opencv-python matplotlib

In [ ]:
# GPUの確認
import torch
if torch.cuda.is_available():
    print(f"🎮 GPU利用可能: {torch.cuda.get_device_name(0)}")
    print(f"メモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPUが利用できません。CPUで実行します（処理が遅くなります）")
    print("ランタイム → ランタイムのタイプを変更 → GPU を選択してください")

In [ ]:
# Google Driveのマウント（生成画像を保存する場合）
from google.colab import drive
drive.mount('/content/drive')

# 保存用ディレクトリの作成
import os
save_dir = '/content/drive/MyDrive/ai_generated_images'
os.makedirs(save_dir, exist_ok=True)
print(f"💾 画像保存先: {save_dir}")

## 1. シンプルな画像生成モデル

まずは軽量なモデルで基本的な画像生成を体験します。

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# 日本語フォントの設定
!apt-get -y install fonts-ipafont-gothic
plt.rcParams['font.family'] = 'IPAGothic'

class SimpleImageGenerator:
    def __init__(self, model_id="CompVis/stable-diffusion-v1-4"):
        """シンプルな画像生成クラス"""
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        print(f"🚀 モデルを読み込み中: {model_id}")
        print("（初回は時間がかかります）")
        
        # パイプラインの初期化
        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            safety_checker=None,
            requires_safety_checker=False
        )
        self.pipe = self.pipe.to(self.device)
        
        # メモリ最適化
        if self.device == "cuda":
            self.pipe.enable_attention_slicing()
        
        print("✅ モデルの準備完了！")
    
    def generate(self, prompt, negative_prompt="", num_images=1, steps=50, guidance_scale=7.5):
        """画像を生成"""
        print(f"\n🎨 生成中: '{prompt}'")
        print(f"ステップ数: {steps}, ガイダンス: {guidance_scale}")
        
        # 生成
        images = self.pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_images,
            num_inference_steps=steps,
            guidance_scale=guidance_scale
        ).images
        
        return images
    
    def display_images(self, images, prompts=None, save=True):
        """画像を表示して保存"""
        n_images = len(images)
        fig, axes = plt.subplots(1, n_images, figsize=(6*n_images, 6))
        
        if n_images == 1:
            axes = [axes]
        
        for i, (img, ax) in enumerate(zip(images, axes)):
            ax.imshow(img)
            ax.axis('off')
            if prompts and i < len(prompts):
                ax.set_title(prompts[i][:30] + "...", fontsize=10)
            
            # 保存
            if save:
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                filename = f"{save_dir}/generated_{timestamp}_{i}.png"
                img.save(filename)
                print(f"💾 保存: {filename}")
        
        plt.tight_layout()
        plt.show()

# ジェネレーターの初期化
generator = SimpleImageGenerator()

## 2. 基本的な画像生成

シンプルなプロンプトで画像を生成してみましょう。

In [ ]:
# 基本的な画像生成
prompts = [
    "a beautiful japanese garden with cherry blossoms",
    "cute robot assistant working in office",
    "futuristic tokyo cityscape at night"
]

for prompt in prompts:
    images = generator.generate(prompt, steps=30)
    generator.display_images(images, [prompt])

## 3. プロンプトエンジニアリング

より詳細な指示でクオリティの高い画像を生成します。

In [ ]:
# プロンプトのテクニック
advanced_prompts = [
    {
        "prompt": "professional product photo of sleek smartphone, minimalist white background, studio lighting, high quality, 8k",
        "negative": "blur, low quality, noise",
        "description": "製品写真スタイル"
    },
    {
        "prompt": "oil painting of mount fuji at sunset, impressionist style, vibrant colors, brush strokes visible",
        "negative": "photo, realistic, digital art",
        "description": "絵画スタイル"
    },
    {
        "prompt": "isometric 3d render of cute miniature factory, pastel colors, soft lighting, artstation",
        "negative": "realistic, photo, dark",
        "description": "3Dアートスタイル"
    }
]

for item in advanced_prompts:
    print(f"\n📝 {item['description']}")
    images = generator.generate(
        prompt=item['prompt'],
        negative_prompt=item['negative'],
        steps=40,
        guidance_scale=8.0
    )
    generator.display_images(images, [item['prompt']])

## 4. インタラクティブな画像生成アプリ

Gradioを使って使いやすいUIを作成します。

In [ ]:
import gradio as gr

def generate_with_ui(prompt, negative_prompt, num_steps, guidance_scale, seed):
    """UI用の生成関数"""
    if seed > 0:
        torch.manual_seed(seed)
    
    images = generator.generate(
        prompt=prompt,
        negative_prompt=negative_prompt,
        steps=num_steps,
        guidance_scale=guidance_scale
    )
    
    return images[0]

# プロンプトの例
example_prompts = [
    ["japanese castle in spring with cherry blossoms, detailed architecture", "blur, low quality", 30, 7.5, -1],
    ["cute anime style character, colorful, happy expression", "realistic, 3d", 40, 8.0, -1],
    ["modern office interior design, minimalist, natural lighting", "dark, cluttered", 35, 7.0, -1],
    ["delicious ramen bowl, food photography, steam, appetizing", "cartoon, illustration", 30, 7.5, -1]
]

# Gradio UI
with gr.Blocks(title="AI画像生成スタジオ") as app:
    gr.Markdown("""
    # 🎨 AI画像生成スタジオ
    
    テキストの説明から画像を生成します。英語でプロンプトを入力してください。
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(
                label="プロンプト（生成したい画像の説明）",
                placeholder="例: beautiful japanese garden with koi pond",
                lines=3
            )
            
            negative_prompt = gr.Textbox(
                label="ネガティブプロンプト（避けたい要素）",
                placeholder="例: blur, low quality, dark",
                lines=2
            )
            
            with gr.Row():
                num_steps = gr.Slider(
                    minimum=10,
                    maximum=50,
                    value=30,
                    step=5,
                    label="生成ステップ数（多いほど高品質）"
                )
                
                guidance_scale = gr.Slider(
                    minimum=1.0,
                    maximum=15.0,
                    value=7.5,
                    step=0.5,
                    label="ガイダンススケール（プロンプトへの忠実度）"
                )
            
            seed = gr.Number(
                label="シード値（-1でランダム）",
                value=-1,
                precision=0
            )
            
            generate_btn = gr.Button("🚀 画像を生成", variant="primary")
            
            # 使い方のヒント
            with gr.Accordion("💡 プロンプトのコツ", open=False):
                gr.Markdown("""
                ### 効果的なプロンプトの書き方
                
                1. **具体的に記述**: "dog" → "golden retriever puppy playing in park"
                2. **スタイルを指定**: "oil painting", "anime style", "photorealistic"
                3. **品質を指定**: "high quality", "detailed", "8k resolution"
                4. **構図を指定**: "close-up", "wide angle", "from above"
                5. **雰囲気を指定**: "dramatic lighting", "peaceful", "vibrant colors"
                
                ### よく使うキーワード
                - アート: illustration, digital art, concept art, artstation
                - 写真: photography, DSLR, bokeh, professional photo
                - 3D: 3d render, octane render, unreal engine, isometric
                """)
        
        with gr.Column(scale=1):
            output_image = gr.Image(
                label="生成された画像",
                type="pil"
            )
            
            # ダウンロードボタンは自動的に表示される
    
    # 例の表示
    gr.Examples(
        examples=example_prompts,
        inputs=[prompt, negative_prompt, num_steps, guidance_scale, seed],
        label="プロンプトの例"
    )
    
    # イベントハンドラー
    generate_btn.click(
        fn=generate_with_ui,
        inputs=[prompt, negative_prompt, num_steps, guidance_scale, seed],
        outputs=output_image
    )

# アプリを起動
app.launch(share=True)

## 5. 画像編集（Image-to-Image）

既存の画像を元に新しい画像を生成します。

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
import requests
from io import BytesIO

class ImageEditor:
    def __init__(self):
        """画像編集用のパイプライン"""
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        print("🔧 画像編集モデルを準備中...")
        self.pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            "CompVis/stable-diffusion-v1-4",
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            safety_checker=None,
            requires_safety_checker=False
        )
        self.pipe = self.pipe.to(self.device)
        
        if self.device == "cuda":
            self.pipe.enable_attention_slicing()
        
        print("✅ 準備完了！")
    
    def edit_image(self, image, prompt, strength=0.75, guidance_scale=7.5):
        """画像を編集"""
        # PILイメージに変換
        if isinstance(image, str):
            if image.startswith('http'):
                response = requests.get(image)
                image = Image.open(BytesIO(response.content))
            else:
                image = Image.open(image)
        
        # リサイズ（512x512に）
        image = image.convert('RGB')
        image = image.resize((512, 512))
        
        # 編集実行
        edited = self.pipe(
            prompt=prompt,
            image=image,
            strength=strength,
            guidance_scale=guidance_scale
        ).images[0]
        
        return edited
    
    def show_before_after(self, original, edited, title=""):
        """編集前後を比較表示"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        
        ax1.imshow(original)
        ax1.set_title("元画像")
        ax1.axis('off')
        
        ax2.imshow(edited)
        ax2.set_title(f"編集後: {title}")
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()

# エディターの初期化
editor = ImageEditor()

In [ ]:
# サンプル画像で編集を試す
from google.colab import files

print("📷 編集したい画像をアップロードしてください")
uploaded = files.upload()

if uploaded:
    # アップロードされた最初の画像を使用
    image_path = list(uploaded.keys())[0]
    original = Image.open(image_path)
    
    # 様々な編集を試す
    edits = [
        {"prompt": "make it cyberpunk style with neon lights", "strength": 0.8},
        {"prompt": "transform to watercolor painting style", "strength": 0.9},
        {"prompt": "make it look like winter scene with snow", "strength": 0.7},
    ]
    
    for edit in edits:
        print(f"\n🎨 編集中: {edit['prompt']}")
        edited = editor.edit_image(
            original,
            prompt=edit['prompt'],
            strength=edit['strength']
        )
        editor.show_before_after(original, edited, edit['prompt'])
else:
    print("画像がアップロードされていません")

## 6. 実践的な活用例

ビジネスや創作活動での活用方法を紹介します。

In [ ]:
# ビジネス向けの画像生成例
business_examples = [
    {
        "category": "プレゼンテーション素材",
        "prompts": [
            "modern business meeting room, clean design, professional atmosphere",
            "abstract technology background, blue gradient, circuit pattern",
            "teamwork illustration, diverse people collaborating, flat design"
        ]
    },
    {
        "category": "製品モックアップ",
        "prompts": [
            "smartphone app mockup, modern UI design, minimalist",
            "eco-friendly packaging design, natural materials, green concept",
            "futuristic product design, sleek metallic finish, innovative"
        ]
    },
    {
        "category": "マーケティング素材",
        "prompts": [
            "happy customer using product, lifestyle photography style",
            "social media banner, vibrant colors, sale promotion",
            "infographic style illustration, data visualization, clean design"
        ]
    }
]

for category in business_examples:
    print(f"\n📊 {category['category']}")
    print("=" * 50)
    
    for prompt in category['prompts']:
        print(f"\n生成中: {prompt[:50]}...")
        images = generator.generate(prompt, steps=30)
        generator.display_images(images, [prompt])

## まとめと今後の学習

このノートブックで学んだこと：
1. AI画像生成の基本的な使い方
2. プロンプトエンジニアリングのテクニック
3. 画像編集とスタイル変換
4. ビジネスでの活用方法

### 📚 さらに学ぶには

1. **高度なモデル**:
   - SDXL (Stable Diffusion XL)
   - Midjourney API
   - DALL-E API

2. **専門的な技術**:
   - LoRA (Low-Rank Adaptation) でのファインチューニング
   - ControlNet での精密な制御
   - カスタムモデルの学習

3. **応用分野**:
   - ゲーム開発のアセット生成
   - ファッションデザイン
   - 建築ビジュアライゼーション

### 🎯 実践課題

1. 自社製品のプロモーション画像を生成
2. ブログ記事用のイラストを作成
3. SNS投稿用のビジュアルコンテンツ制作

AI画像生成は急速に進化している分野です。
定期的に新しい技術やモデルをチェックしましょう！